# Проект "Исследование надежности заемщиков"

## 1. Описание проекта <a id='1'> </a>

**Контекст исследования**:

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования**: определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

**План исследования**:

1) **[Описание проекта](#1)**
2) **[Обзор данных](#2)**
3) **[Предобработка данных](#3)**
4) **[Исследование данных](#4)**
5) **[Вывод](#5)**

**Описание столбцов**:
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## 2. Обзор данных <a id='2'> </a>

In [1]:
#import libraries
import pandas as pd

In [2]:
#read data
try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
#5 random records
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16872,1,-101.861350,28,высшее,0,гражданский брак,1,M,сотрудник,0,101948.053221,сыграть свадьбу
13900,0,-949.943142,29,среднее,1,Не женат / не замужем,4,M,сотрудник,1,138864.225938,строительство собственной недвижимости
19286,0,383930.088898,61,среднее,1,женат / замужем,0,F,пенсионер,0,439443.346784,покупка недвижимости
559,1,-8124.768051,53,среднее,1,вдовец / вдова,2,F,компаньон,0,101876.814932,жилье
5951,1,-5104.078736,42,среднее,1,женат / замужем,0,F,госслужащий,0,370985.730460,на покупку подержанного автомобиля


In [4]:
#main info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert"; style="border-left: 7px solid pink">

**Промежуточный вывод**
- 21525 строк;
- В каждой строке таблицы — данные о одному клиенту банка.;
- Присутствуют отрицительные значения в столбце `days_employed`;
- Присутствуют пропуски в столбцах `days_employed` и `total_income`.
</div>

## 3. Предобработка данных <a id='3'> </a>

### 3.1. Удаление пропусков

In [5]:
#missing values
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

`total_income` хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`:

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

In [7]:
#checking missing values
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### 3.2. Обработка аномальных значений

В данных встречаются аномалии — отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце: заменим все отрицательные значения положительными:

In [8]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведим медианное значение трудового стажа `days_employed` в днях:

In [9]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

Выведем перечень уникальных значений столбца `children`:

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

 Заметим, что в столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`:

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)]
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### 3.3. Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`:

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
#checking missing values
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 3.4. Замена типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`:

In [14]:
data['total_income'] = data['total_income'].astype(int)

### 3.4. Обработка дубликатов

In [15]:
#checking duplicates
data.duplicated().sum()

54

In [16]:
#processing duplicates
data = data.drop_duplicates()

In [17]:
#checking duplicates x2
data.duplicated().sum()

0

### 3.4. Категоризация данных

**Доход**

На основании диапазонов, указанных ниже, создим в датафрейме `data` столбец `total_income_category` с категориями:**
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Цель кредита**

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

<div class="alert"; style="border-left: 7px solid pink">

**Промежуточный вывод**
- Обработали пропуски из столбцов `days_employed` и `total_income`;
- Обработали аномальные значения;
- Удалили дубликаты в столбце с образованием;
- Категоризовали данные по столбцам `total_income` и `purpose`.
</div>

## 4. Исследование данных <a id='4'> </a>

### 4.1. Зависимость между количеством детей и возвратом кредита в срок

In [23]:
def percent_apply(a):
    pivot_table = pd.pivot_table(data, values='debt', index = [a], aggfunc = ['sum', 'count'])
    pivot_table['Доля должников, %'] = (pivot_table['sum'] / pivot_table['count'])
    pivot_table['Доля должников, %'] = pivot_table['Доля должников, %'].apply('{:.2%}'.format)
    pivot_table.columns = ['Всего должников', 'Всего кредитополучателей', 'Процент должников']
    return pivot_table

In [24]:
percent_apply('children')

,Всего должников,Всего кредитополучателей,Процент должников
children,,,
0,1063,14107,7.54%
1,444,4809,9.23%
2,194,2052,9.45%
3,27,330,8.18%
4,4,41,9.76%
5,0,9,0.00%


Данных по клиентам с количество детей 3 и более значительно меньше, чем по другим категориям: 330, 41 и 9 записей соответственно. Это не дает возможности делать однозначные и корректные выводы по ним. Говоря про клиентов с количеством детей 0, 1 и 2 можно сделать вывод что с количеством детей растет доля задолженностей. Следоваельно, в рамках этого количества детей зависимость имеется.

### 4.2. Зависимость между семейным положением и возвратом кредита в срок

In [25]:
percent_apply('family_status')

,Всего должников,Всего кредитополучателей,Процент должников
family_status,,,
Не женат / не замужем,273,2796,9.76%
в разводе,84,1189,7.06%
вдовец / вдова,63,951,6.62%
гражданский брак,385,4146,9.29%
женат / замужем,927,12266,7.56%


Можно заметить, что доля должников среди людей, которые побывали в браке, меньше. Можно предположить, что есть связь с возрастом либо психологическими особенностями. Также наблюдается то, что среди тех, чте находятся в официальном браке, задолденностей больше, что, возможно, связано с меньшем чувством ответственности

### 4.3. Зависимость между уровнем дохода и возвратом кредита в срок

In [26]:
percent_apply('total_income_category')

,Всего должников,Всего кредитополучателей,Процент должников
total_income_category,,,
A,2,25,8.00%
B,354,5014,7.06%
C,1353,15938,8.49%
D,21,349,6.02%
E,2,22,9.09%


В рамках анализа данной сводной таблицы смотрим на категории "B" и "С" - по другим количество записей слишком маленькое для формирования статистически корректных выводов. Исходя из категорий "B" и "С" можно сделать вывод, что среди людей с большим доходом процент своевременно выплаченных кредитов выше.

### 4.3. Влияние цели кредита на его возврат в срок

In [27]:
percent_apply('purpose_category')

,Всего должников,Всего кредитополучателей,Процент должников
purpose_category,,,
операции с автомобилем,400,4281,9.34%
операции с недвижимостью,780,10754,7.25%
получение образования,369,3989,9.25%
проведение свадьбы,183,2324,7.87%


Наиболее надежная цель - "операции с недвижимостью", наиболее рискованные - "операции с автомобилем" и "получение образования"

### 4.4. Возможные причины появления пропусков в исходных данных

В исходных данных пропуски появлялись в двух столбцах:
- в общем трудовом стаже в днях
- в ежемесячном доходе.

Говоря о трудовом стаже, пропуски можно объяснить тем, что человек попросту не работал или же тем, что в банковскую систему не подгрузились данные из пенсионного фонда.

Говоря про среднемесячный доход, можно предположить, что пропуски обусловлены тем, что человек официально не трудоустроен либо же не дал согласие на предоставление банку данных о своих доходах.

Кроме того, можно отметить человеческий и технический факторы. Так, клиенты могли не предоставить данные ввиду того, что эту информацию не требуют в некоторых случаях. Или же проблемы могли возникнуть на этапе загрузки и на этапе выгрузки данных: неправильный запрос, объединение таблиц и т.д.

## 5. Вывод <a id='5'> </a>

**Перед исследования данные были предобработаны, в частности**:
- обработаны пропуски из столбцов `days_employed` и `total_income`
- обработаны аномальные значения
- удалены дубликаты в столбце с образованием
- категоризованы данные по столбцам `total_income` и `purpose`

**В результате исследования были выявлены некоторые зависимости в данных и сформулированы умозаключения, в частности**:
- с количеством детей растет доля задолженностей в данных (распростроняется на данные, где кол-во детей  0, 1 и 2)
- доля должников среди людей, которые побывали в браке, меньше чем среди тех, кто состоит в официальном браке (наибольшая доля должников у людей, никогда не бывавших в официальном браке)
- среди людей с высоким доходом, но до миллиона рублей в месяц, с ростом уменьшается доля просроченных кредитов
- наиболее надежная цель кредита - операции с недвижимостью, наиболее рискованные - операции с автомобилем и получение образования
- пропуски в данных могут быть обсуловлены человеческими, техническими и социально-экономическими факторами
- аномальные значения влияют на репрезентативность и правдивость выводов

**Рекомендации**:
- сформировать другую категоризацию по доходам
- исследовать другие признаки
- рассчитать другие метрики (отношение дохода к сумме выплат и др.)
- сделать ы анкете сделать поля о доходе и стаже обязательными к заполнению, добавить автопроверку на формат вводимых пользователем данных